# Drug Name Entity Classifier
## AHLT - MIRI 2018



## Initialization

Load needed modules and specify the working directory

In [1]:
# Load needed packages
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,KFold # Parameter selection
import time # Execution time of some blocks
from nltk.tag import StanfordPOSTagger
import statistics
import scipy.stats # for RandomizedSearchCV


# Import our defined functions
from NER_functions import *
from datasetBuilder import *

In [2]:
# init time
first_init = time.time()

In [3]:
# Set the data directories
# train_dirs_whereto_parse = ['data/medium_train_DrugBank']
# test_dirs_whereto_parse = ['data/medium_test_DrugBank']
train_dirs_whereto_parse = ['data/Train/DrugBank','data/Train/MedLine']
test_dirs_whereto_parse = ['data/Test/DrugBankOutput','data/Test/MedlineOutput']

## Reading the train and test data from the XML files
Accessing to all the files of the directory and storing id's and text's in two arrays.
We have also added the token 'STOP' at the end of each sentence.

In [4]:
train_texts_entities = createTrainSet(train_dirs_whereto_parse)
test_texts_entities = createTestSet(test_dirs_whereto_parse)

Before computing features, I want the input data to have a special format:

In [5]:
# we want each dataset with the following format: 
# for ['have','Ibuprofeno'] ---- [('hola','V','O'),('Ibuprofeno','NN','B')]

def buildSet(text_entities):
    dataset = []
    for text,drugs in text_entities:
        # tokenizing
        tokenized_sentence = nltk.word_tokenize(text)
        # BIO tagging
        tokens_tags = BIOTagger(text, drugs)
        # POS tagging
        tokens_pos = nltk.pos_tag(tokenized_sentence)

        text_triples = []
        for idx,token in enumerate(tokenized_sentence):
            text_triples.append((token,tokens_pos[idx][1],tokens_tags[idx][1]))
        dataset.append(text_triples)
    
    return dataset

# train_set and test_set are list of list of triples; each list of triples refers to a different sentence
train_set = buildSet(train_texts_entities) 
test_set = buildSet(test_texts_entities)

print('Number of training sentences: ',len(train_set))
print('Num of test sentences: ',len(test_set))

Number of training sentences:  5560
Num of test sentences:  665


## FEATURES 

In [6]:
with(open('data/DrugBank_names_DB.txt', 'r')) as f:
    drugbank_db = f.read().splitlines()
    
def word2features(sent, i, database):
    word = sent[i][0]
    postag = sent[i][1]
    
    # orthographic features
    features = {
    'all_uppercase_letters' : allCaps(word), 
    'initial_capital_letter': initCap(word), 
    'contains_capital_letter' : hasCap(word),
    'single_capital_letter' : singleCap(word),
    'punctuation' : punctuation(word),
    'initial_digit' : initDigit(word),
    'single_digit' : singleDigit(word),
    'letter_and_num' : alphaNum(word),
    'many_numbers' : manyNum(word),
    'contains_real_numbers' : realNum(word),
    'intermediate_dash' : inDash(word),
    'has_digit' : hasDigit(word),
    'is_Dash' : isDash(word),
    'is_roman_letter' : roman(word),
    'is_end_punctuation' : endPunctuation(word),
    'caps_mix' : capsMix(word),
        
    # Morphological information: prefixes/suffixes of lengths from 2 to 5 and word shapes of tokens. 
    'word[-5:]': word[-5:],
    'word[-4:]': word[-4:],    
    'word[-3:]': word[-3:],
    'word[-2:]': word[-2:],
    
    # Domain knowledge
    'contains_drug_sufix': containsSufix(word),
    'contains_drug_prefix': containsPrefix(word),

    # POS tag   
    'postag': postag,
    'postag[:2]': postag[:2],
        
    # Is in DrugBank dataset
    'isInDB':isTokenInDB(word,database)
    }
    
    
    
    # context features
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:unigrams' : num_ngrams(word1,1),
            '-1:bigrams' : num_ngrams(word1,2),
            '-1:trigrams' : num_ngrams(word1,3)
        })
        if i!=1:
            word2 = sent[i-2][0]
            postag2 = sent[i-2][1]
            features.update({
            '-2:postag': postag2,
            '-2:postag[:2]': postag2[:2],
            '-2:unigrams' : num_ngrams(word2,1)})
            if len(word2)>2:
                features.update({
                    '-2:bigrams' : num_ngrams(word2,2),
                    '-2:trigrams' : num_ngrams(word2,3)})
            elif len(word2)==2:
                features.update({
                    '-2:bigrams' : num_ngrams(word2,2)})
                
            
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:unigrams' : num_ngrams(word1,1),
            '+1:bigrams' : num_ngrams(word1,2),
            '+1:trigrams' : num_ngrams(word1,3)
        })
        
        if i<len(sent)-2:
            word2 = sent[i+2][0]
            postag2 = sent[i+2][1]
            features.update({
                '+2:postag': postag2,
                '+2:postag[:2]': postag2[:2],
                '+2:unigrams' : num_ngrams(word2,1)})
            if len(word2)>2:
                features.update({
                    '+2:bigrams' : num_ngrams(word2,2),
                    '+2:trigrams' : num_ngrams(word2,3)})
            elif len(word2)==2:
                features.update({
                    '+2:bigrams' : num_ngrams(word2,2)})
            
    else:
        features['EOS'] = True

    return features


def sent2features(sent,database):
    return [word2features(sent, i,drugbank_db) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
 # remember s is a triple defined as follows: (token, POS tag, BIO tag)
X_train = [sent2features(s,drugbank_db) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_test = [sent2features(s,drugbank_db) for s in test_set]
y_test = [sent2labels(s) for s in test_set]


In [8]:
print(X_train[0][-1])

{'all_uppercase_letters': 0, 'initial_capital_letter': 0, 'contains_capital_letter': 0, 'single_capital_letter': 0, 'punctuation': 0, 'initial_digit': 0, 'single_digit': 0, 'letter_and_num': 0, 'many_numbers': 0, 'contains_real_numbers': 0, 'intermediate_dash': 0, 'has_digit': 0, 'is_Dash': 0, 'is_roman_letter': 0, 'is_end_punctuation': 1, 'caps_mix': 0, 'word[-5:]': '.', 'word[-4:]': '.', 'word[-3:]': '.', 'word[-2:]': '.', 'contains_drug_sufix': 0, 'contains_drug_prefix': 0, 'postag': '.', 'postag[:2]': '.', 'isInDB': False, '-1:postag': 'NNP', '-1:postag[:2]': 'NN', '-1:unigrams': ['E', 'M', 'C', 'Y', 'T'], '-1:bigrams': ['EM', 'MC', 'CY', 'YT'], '-1:trigrams': ['EMC', 'MCY', 'CYT'], '-2:postag': 'IN', '-2:postag[:2]': 'IN', '-2:unigrams': ['o', 'f'], '-2:bigrams': ['of'], 'EOS': True}


In [9]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

####  Predictions

In [10]:
pred_tags = crf.predict(X_test)


####  Evaluation

In [11]:
precision = []
recall = []
for idx, text_drugs in enumerate(test_texts_entities): 
    # so text_drugs will be a tuple of (sentence, list of true entities in that sentence)
    tokens = nltk.word_tokenize(text_drugs[0])
    predicted_entities = BIOTagsToEntities(tokens,pred_tags[idx])
    precision = precision + [compute_precision(predicted_entities,text_drugs[1])]
    recall = recall + [compute_recall(predicted_entities,text_drugs[1])]
    print('tokens: ',tokens)
    print('pred tags: ',pred_tags[idx])
    print('true tags: ',y_test[idx])
    print('pred entities: ',predicted_entities)
    print('true entities:',text_drugs[1],'\n')

avg_precision = statistics.mean(precision)
avg_recall = statistics.mean(recall)

# F1 metric
F1 = round((2*avg_precision*avg_recall) / (avg_precision + avg_recall),2)
print(F1)

tokens:  ['Drug', 'Interactions', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', 'This', 'medicine', 'may', 'be', 'affected', 'by', 'the', 'drug', 'cholestyramine', 'and', 'should', 'not', 'be', 'taken', 'with', 'any', 'form', 'of', 'estrogen', 'therapy', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

tokens:  ['The', 'physiological', 'changes', 'in', 'organ', 'function', 'in', 'older', 'people', 'and', 'the', 'effect', 'of', 'this', 'on', 'pharmacokinetics', 'and', 'pharmacodynamics', 'are', 'discussed', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  []
true entities: [] 

tokens:  ['The', 'risks', 'of', 'adverse', 'drug', 'reactions', 'and', 'adverse', 'drug', 'interactions', 'linked', 'to', 'polypharmacy', 'are', 'explored', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  []
true entities: [] 

tokens:  ['Strategies', 'to', 'achieve', 'optimal', 'prescribing', 'in', 'older', 'people', 'are', 'considered', '.']
pred ta

pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  []
true entities: [] 

tokens:  ['To', 'improve', 'guideline', 'quality', ',', 'more', 'attention', 'should', 'particularly', 'be', 'paid', 'to', 'the', 'available', 'tools', 'for', 'applications', 'and', 'cost', 'implications', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  []
true entities: [] 

tokens:  ['Despite', 'the', 'availability', 'and', 'daily', 'use', 'of', 'computerized', 'drug-drug', 'interaction', 'surveillance', 'systems', ',', 'exposure', 'to', 'potentially', 'relevant', 'drug-drug', 'interactions', '(', 'DDIs', ')', 'continues', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

pred entities:  ['SD']
true entities: [] 

tokens:  ['However', ',', 'the', 'results', 'for', 'group', '1', 'and', 'group', '2', 'were', 'similar', 'after', '14', 'days', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  []
true entities: [] 

tokens:  ['Novel', 'insights', 'into', 'the', 'synergistic', 'interaction', 'of', 'Bortezomib', 'and', 'TRAIL', ':', 'tBid', 'provides', 'the', 'link.', '&', '#', 'xd', ';', '&', '#', 'xa', ';']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  ['Bortezomib', 'TRAIL']
true entities: ['Bortezomib'] 

tokens:  ['The', 'proteasome', 'inhibitor', 'Bortezomib', 'has', 'been', 'id


tokens:  ['Pulmonary', 'tuberculosis', ':', 'clinical', 'features', 'and', 'patient', 'management.', '&', '#', 'xd', ';', '&', '#', 'xa', ';']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  []
true entities: [] 

tokens:  ['Pulmonary', 'tuberculosis', '(', 'TB', ')', 'is', 'a', 'common', 'infectious', 'disease', 'and', 'a', 'major', 'cause', 'of', 'illness', 'and', 'death', 'throughout', 'the', 'world', ',', 'particularly', 'in', 'developing', 'countries', '.']
pred tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
pred entities:  []
true entities: [] 

tokens:  ['This', 'article', 'explores', 't

In [12]:
print('Total execution time: ',(time.time() - first_init)/60, ' minutes')

Total execution time:  1.2746639370918273  minutes


In [13]:

'''
## Log of results
date, precision, recall, F1, features, test
14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes
'''

'\n## Log of results\ndate, precision, recall, F1, features, test\n14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes\n'